# Exploring Crawled Reddit Data

In [1]:

import json
from pathlib import Path

import pandas as pd
# jupyter
from tqdm import tqdm

In [2]:
# some additional configs, pertains to how certain cells are run/behave
write_to_db = True
db_config = {
    'host': '10.20.4.246',
    'port': 3306,
    'user': 'sandman',
    'database': 'cs6471'
}

In [3]:
Path.cwd()

WindowsPath('E:/gatech/courses/cs-6471-computational-social-science/project/utils/reddit')

In [4]:
data_dir = Path.cwd() / 'data/PushShiftAndRedditAPICrawler-output'
data_files = list(data_dir.glob('*.json'))

In [5]:
# load submissions and comments from json files
all_submissions = []
all_comments = []

for f in tqdm(data_files):
    f: Path
    data = json.load(f.open(mode='r', encoding='utf-8'))
    submission = data['submission']['data']

    # data['comments'] = {'type': 't1', 'data': 'actual comment data object'}
    comments = map(lambda c: c['data'], data['comments'])

    all_submissions.append(submission)
    all_comments.extend(comments)

100%|██████████| 20167/20167 [00:07<00:00, 2568.06it/s]


In [6]:
used_columns_df_submissions = [
    'author',
    'author_fullname',
    'created_utc',  # float64
    'name',  # text used as fullname
    'num_comments',  # int64
    'num_crossposts',  # int64
    'num_duplicates',  # int64
    'score',  # int64 num upvotes
    'selftext',  # longtext
    'subreddit_id',  # text fullname of subreddit
    'subreddit_name_prefixed',  # text prefixed name of the subreddit
    'subreddit_subscribers',  # 'int64'
    'subreddit_type',  # text
    'title',  # text
]
df_submissions = pd.DataFrame(all_submissions)[used_columns_df_submissions]
df_submissions.head()

,author,author_fullname,created_utc,name,num_comments,num_crossposts,num_duplicates,score,selftext,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,title
0,[deleted],NaN,1.546344e+09,t3_abhq6i,2,0,0,2,[deleted],t5_2qhlc,r/privacy,1245595,public,Anyone tried Jami Messenger?
1,amboris,t2_god9h,1.546349e+09,t3_abi6je,9,0,0,5,This post will bring me a lot of downvotes but...,t5_2tk0s,r/unpopularopinion,2383260,public,This is why i hate Discord and you should to! :/
2,[deleted],NaN,1.546350e+09,t3_abiah9,43,0,0,221,[deleted],t5_2qh1q,r/india,765796,public,Ola's Privacy Policy is creepy
3,EndMeetsEnd,t2_rln9c,1.546376e+09,t3_abltgc,0,0,1,1,,t5_skup3,r/TheLibertarianProject,2,public,Data Privacy Scandals and Public Policy Pickin...
4,Armane407,t2_1alkf8uj,1.546376e+09,t3_ablxx1,1,0,0,1,\n\n￼\n\nLATEST NEWS\n\nFirewall Zero Hour Dev...,t5_9krdo,r/FireWallZeroHour,5447,public,Nice to know


In [7]:
used_columns_df_comments = [
    'author',
    'author_fullname',
    'body',  # longtext content of the comment
    'controversiality',  # float64
    'created_utc',  # float64
    'depth',  # int
    'name',  # text fullname of the comment
    'parent_id',  # text fullname of the parent
    'score',  # int64
    'subreddit_id',  # text fullname of the subreddit
    'subreddit_name_prefixed',
]
df_comments = pd.DataFrame(all_comments)[used_columns_df_comments]
df_comments.head()

,author,author_fullname,body,controversiality,created_utc,depth,name,parent_id,score,subreddit_id,subreddit_name_prefixed
0,[deleted],NaN,[deleted],0.0,1.546554e+09,0,t1_ed6jj48,t3_abhq6i,5.0,t5_2qhlc,r/privacy
1,tedkotz,t2_hnvc9,They are not a service provider. That is proba...,0.0,1.554926e+09,0,t1_ekknevu,t3_abhq6i,1.0,t5_2qhlc,r/privacy
2,AutoModerator,t2_6l4z3,Hi everyone! Please make sure to **upvote** we...,0.0,1.546349e+09,0,t1_ed0ev41,t3_abi6je,1.0,t5_2tk0s,r/unpopularopinion
3,Flamingpanda2000,t2_1bd437bt,Nobody’s going to downvote your opinion becaus...,0.0,1.546350e+09,0,t1_ed0f60a,t3_abi6je,7.0,t5_2tk0s,r/unpopularopinion
4,Vic9420,t2_mlbi9,"I use Discord,but only as a way to chat with a...",0.0,1.546350e+09,0,t1_ed0f89z,t3_abi6je,2.0,t5_2tk0s,r/unpopularopinion


In [14]:
if write_to_db:
    # get a database connection
    import sqlalchemy
    import sqlalchemy.dialects.mysql
    from sqlalchemy.dialects.mysql import TEXT, LONGTEXT, BIGINT, DOUBLE

    print(f'Using SQL DB, config: {db_config}')
    engine = sqlalchemy.create_engine(
        f'mysql+mysqlconnector://{db_config["user"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}',
        echo=False)
    print(vars(engine))
    print('Writing submissions dataframe to DB')

    # self text is very long, we need to explicitly set this to longtext
    df_submissions.to_sql(
        name='submissions',
        con=engine,
        if_exists='replace',
        method='multi',
        chunksize=512,
        dtype={
            'author': TEXT,
            'author_fullname': TEXT,
            'created_utc': BIGINT,  # float64
            'name': TEXT,  # text used as fullname
            'num_comments': BIGINT,  # int64
            'num_crossposts': BIGINT,  # int64
            'num_duplicates': BIGINT,  # int64
            'score': BIGINT,  # int64 num upvotes
            'selftext': LONGTEXT,  # longtext
            'subreddit_id': TEXT,  # text fullname of subreddit
            'subreddit_name_prefixed': TEXT,  # text prefixed name of the subreddit
            'subreddit_subscribers': BIGINT,  # 'int64'
            'subreddit_type': TEXT,  # text
            'title': TEXT,  # text
        }
    )
    print(f'Writing comments to DB')
    # body is very long, we need to explicitly set this to longtext
    df_comments.to_sql(
        name='comments',
        con=engine,
        if_exists='replace',
        method='multi',
        chunksize=512,
        dtype={
            'author': TEXT,
            'author_fullname': TEXT,
            'body': LONGTEXT,  # longtext content of the comment
            'controversiality': DOUBLE,  # float64
            'created_utc': BIGINT,  # float64
            'depth': BIGINT,  # int
            'name': TEXT,  # text fullname of the comment
            'parent_id': TEXT,  # text fullname of the parent
            'score': BIGINT,  # int64
            'subreddit_id': TEXT,  # text fullname of the subreddit
            'subreddit_name_prefixed': TEXT,
        }
    )

Using SQL DB, config: {'host': '10.20.4.246', 'port': 3306, 'user': 'sandman', 'database': 'cs6471'}
{'pool': <sqlalchemy.pool.impl.QueuePool object at 0x000001A66C5F4448>, 'url': mysql+mysqlconnector://sandman@10.20.4.246:3306/cs6471, 'dialect': <sqlalchemy.dialects.mysql.mysqlconnector.MySQLDialect_mysqlconnector object at 0x000001A66C4562C8>, '_echo': False, 'logger': <Logger sqlalchemy.engine.Engine (WARNING)>, 'hide_parameters': False, '_compiled_cache': {}}
Writing submissions dataframe to DB
Writing comments to DB


In [ ]:
df_submissions.iloc[0]['user_reports'].dtype

In [11]:
df_submissions['selftext'].astype(str, inplace=True)

TypeError: astype() got an unexpected keyword argument 'inplace'